In [2]:
from external.tab_pfn_gen.src.tabpfgen.tabpfgen import TabPFGen
import torch
from performance_metrics.measure_model_performance import measure_model_clasification_performance, measure_distance_to_nearest_neighbour
from performance_metrics.measure_model_performance import  measure_regresion_model_performance, measure_k_anonimity
import pandas as pd

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


# Test quality

In [ ]:
generator = TabPFGen(n_sgld_steps=200, device=device)
print(measure_model_clasification_performance(generator.generate_classification))

In [ ]:
generator = TabPFGen(n_sgld_steps=200, device=device)
print(measure_regresion_model_performance(generator.generate_regression))

In [ ]:
generator = TabPFGen(n_sgld_steps=100, device=device)
print(measure_distance_to_nearest_neighbour(generator.generate_regression, 1))

In [ ]:
generator = TabPFGen(n_sgld_steps=200, device=device)
print(measure_k_anonimity(generator.generate_regression))

# FullTabPFGen results

In [7]:
feature_order_results_files = {"FullTabPFGen tree based": "tree.csv", "FullTabPFGen xgb based": "xgb.csv", "FullTabPFGen graph based": "graph.csv", "FullTabPFGen correlation_based": "corelation.csv", "FullTabPFGen": "FulltabPFGen.csv", "TabPFGen": "tabPFGen.csv",  "TabICLGen": "tabPFGen_ICL.csv", "SMOTE": "smote.csv", "Real": "real_data.csv", "CTGAN": "ctgan.csv"}

In [8]:
generation_results = {}

for order_type, result_file in feature_order_results_files.items():
    results = pd.read_csv(result_file, index_col=0)
    generation_results[order_type] = results

In [9]:
display(pd.DataFrame({ordering_type: results.drop(["dataset"], axis=1).mean() for ordering_type, results in generation_results.items()}).T)

,LR,TabICL,TabPFN,random_forest,xgboost
FullTabPFGen tree based,0.891558,0.943673,0.941843,0.909762,0.879491
FullTabPFGen xgb based,0.946173,0.907171,0.933676,0.843965,0.835296
FullTabPFGen graph based,0.911119,0.899419,0.931837,0.911469,0.886160
FullTabPFGen correlation_based,0.946392,0.955507,0.953054,0.898129,0.931822
FullTabPFGen,0.954713,0.938880,0.944661,0.948593,0.901345
TabPFGen,0.936373,0.950484,0.962406,0.945520,0.931265
TabICLGen,0.964618,0.963242,0.963242,0.923735,0.939435
SMOTE,0.958756,0.966243,0.966738,0.938109,0.939495
Real,0.986703,0.979680,0.994592,0.987468,0.984125
CTGAN,0.659687,0.639809,0.669221,0.595880,0.608255
